In [1]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd


In [2]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A_3.csv")
pi = pd.read_csv("pi_3.csv")

A = np.array(A)
pi = np.array(pi)
pi = pi.T
b = np.array([1000 for _ in range(m)])  


In [3]:
def dynamicLearning(start, k, xp, xk):
	Anow = A[:, 0:k]
	Anow = Anow.T
	# print(Anow.shape)
	piNow = (pi.T)[0:k]
	piNow = np.reshape(piNow, (k,))
	bNow = b*(float(k/n))
	bNow = bNow.T

	temp = m+k
	y = cp.Variable(temp)
	e = np.ones(k)
	objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
	constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
	prob = cp.Problem(objective, constraints)

	prob.solve()
	P = y[:m].value
	print(P)
	AT = A.T

	xpNow = xp
	xkNow = xk
	for i in range(start, 2*k):
		if i >= n:
			# print(xkNow)
			global xTrue
			xTrue = xkNow
			return xkNow
		
		cri = (AT[i].dot(P))
		if(pi[0][i] > cri):
			xpNow.append(1)
		else:
			xpNow.append(0)

		test = 1
		for j in range(m):
			temp1 = A[j,i]*xpNow[i]
			temp2 = b[j] - (A[j:j+1, :i]).dot(np.array(xkNow))
			# print(i)
			# print(len(xkNow))
			if temp1 > temp2:
				test = 0
				break
		if test == 1:
			xkNow.append(xpNow[i])
		else:
			xkNow.append(0)

	dynamicLearning(2*k, 2*k, xpNow, xkNow)


In [4]:
xk = [0]*50
xp = [0]*50
xTrue = []

tt = dynamicLearning(50, 50, xp, xk)

[1.35081704 2.0091937  2.8204548  4.17300415 5.10632907 6.00617544
 6.86463668 7.98026771 9.07001817 9.94393725]
[1.23488944 1.89667522 2.78963516 4.23203398 4.99708955 6.00553983
 7.01494116 8.20822609 9.18253666 9.94101562]
[1.21109318 2.04822188 2.81572949 4.2145155  5.09514834 6.11409644
 7.03873742 8.02835596 9.05932856 9.99874148]
[ 1.12814012  2.07698507  2.98519526  4.04922483  5.08651818  6.04798621
  7.10067256  8.04116123  9.0261208  10.11844939]
[ 1.07795779  2.0624186   3.0539594   4.0778919   5.11204877  6.04852261
  7.11470263  8.06321288  8.97962325 10.09986758]
[ 1.03875588  2.04992728  3.06255416  4.13007481  5.07947292  6.07076231
  7.08960241  8.03802056  9.01581167 10.09524828]
[ 1.05735192  2.03676358  3.06308278  4.0870983   5.07464867  6.04986122
  7.07681815  8.08514301  9.06491709 10.07973132]
[ 1.04582533  2.07832573  3.0754302   4.06576602  5.0873529   6.06736535
  7.05443165  8.07984351  9.07065583 10.07619772]


In [5]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()

opv1 = prob.value

In [7]:
print(xTrue)
xTrue = np.array(xTrue)
opv2 = (pi.dot(xTrue))[0]
ratio = opv2/opv1
ratio

[0 0 0 ... 0 0 0]


0.9846025081223694